In [1]:
from datasets import load_dataset

dataset_original = load_dataset("theatticusproject/cuad-qa")

In [2]:
# import spacy
import re
from datasets import DatasetDict
import os

num_proc=os.cpu_count()

from more_itertools import windowed
# nlp = spacy.load("en_core_web_sm")

# def chunk_document(document):
#     doc = nlp(document)
#     sentences = [sent.text for sent in doc.sents]
#     return sentences

def process_record(record):
    document_id = "REFERENCE"
    document = record['context']
    document = re.sub(r'\n+', '\n', document)
    answers = record['answers']
    answers = record['answers']
    if len(answers['text']) > 0:
        gold_text = "Highlights:\n" + "\n".join([f"- {answer}" for answer in answers['text']])
    else:
        gold_text = "No relevant information found in the document."
    citations = [[document_id, document]]
    # sentences = chunk_document(document)
    # oracle_documents_passages = []
    # # max limit of chunk - tells us number of words, fill in the sentences until max is reached
    # for sentence in windowed(sentences, 5, fillvalue="", step=2):
    #     section = " ".join([s for s in sentence if (s is not None and len(s.strip()) > 0)])
    #     oracle_documents_passages.append([document_id, section])
    
    return {
        'docid': str(record['id']),
        'previous_text': record['question'],
        'gold_text': gold_text,
        'citations': citations,
        # 'oracle_documents_passages': oracle_documents_passages,
    }

dataset_train = dataset_original['train'].select(range(1000))
dataset_test = dataset_original['test']

dataset = DatasetDict({
    'train': dataset_train,
    'test': dataset_test,
})
dataset = dataset.map(process_record, num_proc=num_proc)

In [3]:
dataset = dataset.select_columns(['docid', 'previous_text', 'gold_text', 'citations'])
dataset.push_to_hub('ylkhayat/CUAD-generation-workshop', data_dir='data')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ylkhayat/CUAD-generation-workshop/commit/af8243ba9fd5967110189c24dcdb8e1e30036142', commit_message='Upload dataset', commit_description='', oid='af8243ba9fd5967110189c24dcdb8e1e30036142', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ylkhayat/CUAD-generation-workshop', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ylkhayat/CUAD-generation-workshop'), pr_revision=None, pr_num=None)

In [ ]:
# - complete documents
# text is extracted from context

# - align score (gen and the answer) correctness
#  ( gen and top k) faithfulness
 
#  - we need to break the context into chunks (paragraphs) - normal spacy -> to get sentences and then put in paragraph 
 
 
 
#  - instruct to mention no answer explicitly (challenge)
 
#  - challenge correctness (for no answer, manually state that there is no answer for better scores)
#  - challenge faithfulness (exclude no answers OR base don generations (if exists evaluate faithfulness, otherwise exclude the record))